### PDF Knowledge Graph

![rag flow image](graph-db.png "RAG FLOW")

### Chat application with Mem0 (vector + graph)

![rag flow image](chat-application.png "RAG FLOW")

In [1]:
import os
from dotenv import load_dotenv
from mem0 import Memory
from openai import OpenAI

load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.environ.get('OPEN_API_KEY')
MEM0_API_KEY = os.environ.get('MEM0_API_KEY')
NEO4J_URL = "bolt://localhost:7687"
NEO4J_URI = os.environ.get('NEO4J_URI')
NEO4J_USERNAME = os.environ.get('NEO4J_USERNAME')
NEO4J_PASSWORD = os.environ.get('NEO4J_PASSWORD')

In [3]:
config = {
    "version": "v1",
    "embedder": {
        "provider": "openai",
        "config": {
            "api_key": OPENAI_API_KEY,
            "model": "text-embedding-3-small"
        }
    },
    "llm": {
        "provider": "openai",
        "config": {
            "api_key": OPENAI_API_KEY,
            "model": "gpt-4.1"
        }
    },
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "host": "localhost",
            "port": 6333
        }
    },
    "graph_store": {
        "provider": "neo4j",
        "config": {
            "url": NEO4J_URL,
            "username": NEO4J_USERNAME,
            "password": NEO4J_PASSWORD
        }
    },
}

In [4]:
mem_client = Memory.from_config(config)

/Users/rohangore/Documents/projects/python/venv/lib/python3.13/site-packages/pydantic/v1/typing.py:68: DeprecationWarning: Failing to pass a value to the 'type_params' parameter of 'typing.ForwardRef._evaluate' is deprecated, as it leads to incorrect behaviour when calling typing.ForwardRef._evaluate on a stringified annotation that references a PEP 695 type parameter. It will be disallowed in Python 3.15.
  return cast(Any, type_)._evaluate(globalns, localns, recursive_guard=set())
/Users/rohangore/Documents/projects/python/venv/lib/python3.13/site-packages/pydantic/v1/typing.py:68: DeprecationWarning: Failing to pass a value to the 'type_params' parameter of 'typing.ForwardRef._evaluate' is deprecated, as it leads to incorrect behaviour when calling typing.ForwardRef._evaluate on a stringified annotation that references a PEP 695 type parameter. It will be disallowed in Python 3.15.
  return cast(Any, type_)._evaluate(globalns, localns, recursive_guard=set())


In [7]:
def chat(message):
    """
    1. It will do vector embedding of the message.
    2. It will search the vector store for the most similar vectors.
    3. It will then find that vector in the graph store to get nodes and relations(edges).
    """
    mem_result = mem_client.search(query = message, user_id="user_1")
    
    print("Memory: ", mem_result)
    
    # Creating memories from the search results and keep on appeding memory along with score so that it can be used for system context.
    memories = ""
    for memory in mem_result.get('results'):
        memories += f"{str(memory.get('memory'))}:{str(memory.get('score'))}"
        
    SYSTEM_PROMPT = f"""
    You are an memory aware fact extraction agent, an advanced AI designed to systematically analyse input content, extract structured knowledge and maintain an optimized memory store. Your primary function is information distillation and knowledge preservation with contextual awareness.
    
    MEMORY AND SCORING:
    memories: {memories}
    """
    
    messages = [
        {"role":"system", "content": SYSTEM_PROMPT},
        {"role":"user", "content": message},
    ]
    
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    response = openai_client.chat.completions.create(
        model="gpt-4.1",
        messages=messages
    )
    
    messages.append({"role":"assistant", "content": response.choices[0].message.content})
    
    # Adding chat history to memory using mem0
    mem_client.add(messages, user_id="user_1") # user_id can be uuid.
    
    return response.choices[0].message.content

In [8]:
while True:
    message = input(">>")
    print("BOT: ", chat(message))

Memory:  {'results': [{'id': '23f43cde-b438-49a2-ad67-7c8023789bf4', 'memory': 'Name is Rohan Gore', 'hash': '86dd4db7ab044fe0e054a0fcfa548b55', 'metadata': None, 'score': 0.22167593, 'created_at': '2025-04-24T17:32:10.471186-07:00', 'updated_at': None, 'user_id': 'user_1'}, {'id': '499be0d9-3624-4f05-a2f2-4773f0065173', 'memory': 'Likes gym', 'hash': 'ec3c988b1086baca60ea7d69b3790ae1', 'metadata': None, 'score': 0.17482793, 'created_at': '2025-04-24T17:30:26.348411-07:00', 'updated_at': None, 'user_id': 'user_1'}, {'id': 'b5c44bc6-4794-41f7-88b6-7c1e6f768e15', 'memory': 'Wants to eat clean food', 'hash': '978ff8c4cb022a71026386dc0737cd28', 'metadata': None, 'score': 0.14967915, 'created_at': '2025-04-24T17:34:04.906914-07:00', 'updated_at': None, 'user_id': 'user_1'}, {'id': '633a6957-30de-4821-a0e2-f341d6ecdc88', 'memory': 'Wants a lean physique', 'hash': '33909a22a0f9f4a112a62a77255fbe7c', 'metadata': None, 'score': 0.1415332, 'created_at': '2025-04-24T17:34:04.890888-07:00', 'updat

KeyboardInterrupt: 